## Rodando o Modelo

É necessário utilizar os arquivos de jogos e de cadastro na base de dados.

In [1]:
import numpy as np
import pandas as pd

Em primeiro lugar, vamos definir nossas funções que vão atualizar o rating dos times de acordo com os resultados das partidas.

In [2]:
#Funções Elo

def rate_table(entrada):    
    ratings = []
    i=1
    for line in entrada:
        while i!=line[0]:
            ratings.append([i,0.0])
            i+=1
        rate = [0, 0.0]
        s = [line[0],line[1]]
        #s = line.split()
        rate[0]=int(s[0])
        rate[1]=float(s[1])
        ratings.append(rate)
        i+=1
    print(ratings)
    return ratings


def elo(Pb,W,We,I,dif):
    return Pb + I*(W-We)*dif**(1/2)


def elo_rate(ratings, jogos):
    for line in jogos:
        s = [line[0],line[1],line[2],line[3]]
        #divide o registro de um jogo em suas informações: time1, gols1, gols2 e time2
        jogo = [0,0,0,0]        
        jogo[0] = int(s[0])
        jogo[1] = int(s[1])  
        jogo[2] = int(s[2])  
        jogo[3] = int(s[3])
        I=20       
        t1 = jogo[0]
        t2 = jogo[3]
        Pb1 = ratings[t1-1][1]
        Pb2 = ratings[t2-1][1]
        W1 = 0.5
        W2 = 0.5
        dif = jogo[1]-jogo[2]
        if dif>0:
            W1,W2 = 1,0
        elif dif<0:
            W1,W2,dif = 0,1,-dif
        else:
            dif=1
        We1 = 1/(10**((Pb2-Pb1+100)/400)+1)
        We2 = 1/(10**((Pb1-Pb2-100)/400)+1)
        x1 = elo(Pb1,W1,We1,I,dif)
        x2 = elo(Pb2,W2,We2,I,dif)
        ratings[t1-1][1]=x1
        ratings[t2-1][1]=x2
    return ratings





Já temos nossas funções Elo, a lista de partidas e a base de times. Parece que estamos prontos para calcular os ratings. Ou não?

Considerando que os algoritmos acima apenas atualizam os ratings em função dos resultados e ratings passados como argumento, devemos nos perguntar: que ratings devemos considerar no início desta computação?

Nossa base tem partidas desde 2002. Devemos, então, considerar 2002 como "o início dos tempos", de modo que todos os times comecem com o mesmo rating?

Vejamos o que pode ocorrer nesta situação...

Imagine a existência de três campeonatos: o nacional A, o nacional B e o continental C, que inclui times dos países A e B.

Agora imagine que o nacional A conte com 3 times A1, A2 e A3, e que o nacional B conte com 3 times B4, B5 e B6.

Vamos supor ainda que em 3 anos o time A1 ganhou com folgas o nacional A em todas as edições, enquanto no nacional B cada time foi campeão em uma das edições.

Finalmente, vamos supor ainda que no continental C, em cada um dos 3 anos, o campeão de A enfrentou o campeão de B e os jogos tiveram os seguintes placares:

Ano 1: A1 0 x 4 B4

Ano 2: A1 0 x 4 B5

Ano 3: A1 0 x 4 B6

Parece claro que, embora hegemônico no nacional A, A1 tem bastante dificuldade em jogar contra times que disputam o nacional B, tendo sido goleado em 3 de 3 partidas. Seria irreal imaginar o rating de A1 superior aos ratings de B4, B5 e B6 dada essa amostra de jogos, não?

Vamos então simular o resultado dos demais jogos e calcular os ratings para o cenário proposto.

In [3]:
times_fict = ['A1', 'A2', 'A3', 'B4', 'B5', 'B6']
A1, A2, A3, B4, B5, B6 = 1,2,3,4,5,6

rates_fict = [[A1,1500],[A2,1500],[A3,1500],[B4,1500],[B5,1500],[B6,1500]]

jogos_fict = [[]]*39

jogos_fict[0] = [A1, 2, 0, A2]
jogos_fict[1] = [A2, 1, 2, A1]
jogos_fict[2] = [A1, 2, 0, A3]
jogos_fict[3] = [A3, 2, 2, A1]
jogos_fict[4] = [A2, 2, 0, A3]
jogos_fict[5] = [A3, 1, 0, A2]
jogos_fict[6] = [B4, 2, 0, B5]
jogos_fict[7] = [B5, 1, 2, B4]
jogos_fict[8] = [B4, 2, 0, B6]
jogos_fict[9] = [B6, 2, 2, B4]
jogos_fict[10] = [B5, 2, 0, B6]
jogos_fict[11] = [B6, 1, 0, B5]
jogos_fict[12] = [A1, 0, 4, B4]
jogos_fict[13] = [A1, 2, 0, A2]
jogos_fict[14] = [A2, 1, 2, A1]
jogos_fict[15] = [A1, 2, 0, A3]
jogos_fict[16] = [A3, 2, 2, A1]
jogos_fict[17] = [A2, 2, 0, A3]
jogos_fict[18] = [A3, 1, 0, A2]
jogos_fict[19] = [B5, 2, 0, B6]
jogos_fict[20] = [B6, 1, 2, B5]
jogos_fict[21] = [B5, 2, 0, B4]
jogos_fict[22] = [B4, 2, 2, B5]
jogos_fict[23] = [B6, 2, 0, B4]
jogos_fict[24] = [B4, 1, 0, B6]
jogos_fict[25] = [A1, 0, 4, B5]
jogos_fict[26] = [A1, 2, 0, A2]
jogos_fict[27] = [A2, 1, 2, A1]
jogos_fict[28] = [A1, 2, 0, A3]
jogos_fict[29] = [A3, 2, 2, A1]
jogos_fict[30] = [A2, 2, 0, A3]
jogos_fict[31] = [A3, 1, 0, A2]
jogos_fict[32] = [B6, 2, 0, B4]
jogos_fict[33] = [B4, 1, 2, B6]
jogos_fict[34] = [B6, 2, 0, B5]
jogos_fict[35] = [B5, 2, 2, B6]
jogos_fict[36] = [B4, 2, 0, B5]
jogos_fict[37] = [B5, 1, 0, B4]
jogos_fict[38] = [A1, 0, 4, B6]

rates_fict = elo_rate (rates_fict, jogos_fict)

for i in range(6):
    print(times_fict[i], 'rate:', rates_fict[i][1])


A1 rate: 1550.252833428853
A2 rate: 1449.3340432030684
A3 rate: 1452.328409661481
B4 rate: 1503.4745473660973
B5 rate: 1515.6736135442466
B6 rate: 1528.9365527962539


O que queremos dizer com isso?

Ora, a ideia de um ranking é que os times bem colocados devem ser melhores que os times mal colocados. Em particular, se um time tem rating muito melhor do que outro, ele deve ser considerado favorito num confronto direto entre os dois.

Na simulação anterior, o time A1 é hegemônico em seu país, o que justifica um rating bem maior que seus adversários nacionais (1550 contra 1449 e 1452). Por outro lado, A1 foi goleado em todos os confrontos contra times do país B e ainda assim tem rating maior que todos eles (1550 contra 1503, 1515 e 1528), o que contradiz a nossa expectativa de funcionamento desse rating.

O motivo dessa distorção é que, muito provavelmente, os times do país B são melhores que os times do país A. Como todos foram inicializados com o mesmo rating e o volume de jogos é muito maior dentro das competições nacionais, os times que se sobressaem localmente tendem a ter ratings maiores, mesmo sem alcançarem sucesso em competições continentais.

Para evitar que isso ocorra, devemos propor uma métrica para que os times sejam inicializados conforme sua força relativa. Ou, pelo menos, conforme a força relativa dos times do seu país.

Uma primeira abordagem poderia ser analisar os dados e verificar que times ou que países se sobressaíram nos confrontos internacionais.

O gráfico abaixo pode nos dar uma intuição a esse respeito.

<img src="data/aproveitamento_por_pais.png">



Já a tabela abaixo mostra os clubes com maior aproveitamento de resultados, considerando jogos nacionais e internacionais:

<img src="data/aproveitamento_clubes.png">



Como podemos perceber, não há uma correlação clara entre aproveitamento de clubes de um país em jogos internacionais e aproveitamento dos melhores clubes daquele país de modo geral. Isso pode ser explicado tanto pela maior massa de jogos locais como pela variabilidade do equilíbrio de forças entre os clubes dentro do próprio país.

Qual das duas informações parece mais relevante? O aproveitamento dos clubes de um país, em geral, ou o aproveitamento individual de cada clube? Que peso atribuir a cada uma dessas informações?

Além dessas dificuldades técnicas, há ainda um problema conceitual nessa abordagem. 

O rating inicial deveria refletir quão bom um time (ou um país) era em 2002. Ao utilizar nossos dados, com resutados de 2002 a 2019, como proxy pro rating inicial, estaremos "duplicando" a importância desses dados para o modelo, o que deve gerar um outro tipo de distorção.

Deve haver uma outra maneira de estimar os ratings iniciais...

Uma ideia alternativa seria coletar dados anteriores a 2002 para fazer essa estimação. Mas...se esses dados estão disponíveis, por que não incorporamos ao nosso dataset desde o início?

Na prática, não temos como coletar esses dados, neste momento, de forma estruturada. Mas, de forma improvisada, seria possível coletar uma pequena parte desses dados anteriores, para melhorar nossa estimativa inicial.

Com algum esforço e resiliência, coletamos manualmente os resultados da Libertadores de 1999, 2000 e 2001, edições imediatamente anteriores ao início da nossa base de dados.

Como esses dados não são muitos, fica inviável estimar o rating inicial de cada clube individualmente. A abordagem seguida foi "transformar" cada time numa instância do seu país. Assim, todos os jogos entre times de países distintos foram armazenados na planilha lib_9901MEX.xlsx. 

Além disso, criamos uma mini base de times com os países envolvidos nessas edições da Libertadores e inicializamos todos eles com o mesmo rating 1500.

Com isso, atualizamos os ratings desses países de acordo com a lista de jogos.


In [4]:
#EloInicializa.py

#dataframes dos resultados das Libertadores de 1999 a 2001
df_jogosid = pd.read_excel("data/lib_9901MEX.xlsx")
df_teams = pd.read_csv("data/teams99.csv")
df_teams02 = pd.read_csv("data/teams1500.csv")

entrada = list(zip(df_teams.id,df_teams.rate))
ratings = rate_table(entrada) 

jogos = list(zip(df_jogosid.id1,df_jogosid.g1,df_jogosid.g2,df_jogosid.id2))
ratings = elo_rate (ratings, jogos)

#calcula o Elo rate por país referente às Libertadores de 1999 a 2001
rates = [ratings[i][1] for i in range(len(ratings))]
ratings=[]
for rate in rates:
    if rate!=0:
        ratings.append(rate)
df_teams.rate = ratings


[[1, 1500.0], [2, 1500.0], [3, 1500.0], [4, 1500.0], [5, 1500.0], [6, 1500.0], [7, 1500.0], [8, 1500.0], [9, 1500.0], [10, 1500.0], [11, 1500.0]]


Ao fim dessas três edições, os países tinham os seguintes ratings:

In [5]:
print(df_teams.sort_values(by=['rate'],ascending=False)[['id','nome','pais','rate']])

    id       nome       pais         rate
2    3     Brasil     Brasil  1661.602040
10  11     Mexico     Mexico  1639.571552
0    1  Argentina  Argentina  1607.029249
4    5   Colombia   Colombia  1581.678606
8    9    Uruguai    Uruguai  1522.613112
6    7   Paraguai   Paraguai  1458.792976
5    6    Equador    Equador  1445.891332
9   10  Venezuela  Venezuela  1439.130774
3    4      Chile      Chile  1437.392399
1    2    Bolivia    Bolivia  1395.953210
7    8       Peru       Peru  1310.344748


Cabe aqui ressaltar que embora os jogos dos times mexicanos tenham sido excluídos da nossa base original (2002-2019), tivemos que mantê-los neste exercício, uma vez que eles representavam mais de 60% dos jogos dos times venezuelanos.

A razão para tal é que nessa época a competição começava com uma fase eliminatória em que 2 times mexicanos e 2 times venezuelanos se enfrentavam e os 2 melhores colocados avançavam para a fase de grupos.

Embora não tenhamos retirado os times mexicanos para não prejudicar a avaliação dos times venezuelanos, tampouco temos interesse em manter o México na lista, uma vez que não faz parte da América do Sul e seus times não mais disputam as competições deste continente.

Assim, retiramos o México da lista final encontrada, redistribuindo seus pontos uniformemente entre os outros países, de modo a manter a média em 1500.

A nova lista ficou assim:

In [6]:
#renormaliza os ratings para média 1500 sem os times mexicanos
for i in range(len(ratings)-1):
    ratings[i]+=(ratings[10]-1500)/10
ratings[10]=1500

for i in range(10):
    print(df_teams.pais[i], np.round(ratings[i],2))
    
print(np.mean(ratings))



Argentina 1620.99
Bolivia 1409.91
Brasil 1675.56
Chile 1451.35
Colombia 1595.64
Equador 1459.85
Paraguai 1472.75
Peru 1324.3
Uruguai 1536.57
Venezuela 1453.09
1500.0


Por fim, para que nossos ratings iniciais mantenham a média de 1500 pontos, vamos precisar ponderar a pontuação de cada país pelo número de times daquele país na base.

Assim, para países com mais times na base, o rating inicial será aproximado à média desejada de 1500. 

In [7]:
#calcula os número de times cadastrados por país
timesporpais = []
for i in range(10):
    qtd = df_teams02[df_teams02['id']>=100*i][df_teams02['id']<100*(i+1)].id.size
    timesporpais.append(qtd)
    
print(timesporpais)

#pondera o rating de cada país pelo número de times daquele país
ratings_ini = [round(1500+(ratings[i]-1500)*np.min(timesporpais)/timesporpais[i],8) for i in range(10)]
rates_ini = []
for i in range(10):
    for j in range(timesporpais[i]):
        rates_ini.append(ratings_ini[i])

#atribui o respectivo valor inicial encontrado para todos os times da base
df_teams02.rate = rates_ini
cols = ['id','nome','pais','rate']
df_teams02 = df_teams02[cols]
 
#salva o dataframe num .csv
df_teams02.to_csv('data/teams.csv')


[42, 30, 42, 33, 30, 25, 27, 41, 34, 37]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [8]:
print(df_teams02, round(np.mean(df_teams02.rate),8))

      id                  nome       pais         rate
0      1              Aldosivi  Argentina  1572.015717
1      2              All Boys  Argentina  1572.015717
2      3               Almagro  Argentina  1572.015717
3      4        Argentinos Jrs  Argentina  1572.015717
4      5       Arsenal Sarandi  Argentina  1572.015717
5      6          Atl. Rafaela  Argentina  1572.015717
6      7          Atl. Tucuman  Argentina  1572.015717
7      8              Banfield  Argentina  1572.015717
8      9              Belgrano  Argentina  1572.015717
9     10          Boca Juniors  Argentina  1572.015717
10    11       Central Cordoba  Argentina  1572.015717
11    12     Chacarita Juniors  Argentina  1572.015717
12    13        Colon Santa FE  Argentina  1572.015717
13    14     Crucero del Norte  Argentina  1572.015717
14    15    Defensa y Justicia  Argentina  1572.015717
15    16      Estudiantes L.P.  Argentina  1572.015717
16    17        Gimnasia Jujuy  Argentina  1572.015717
17    18  

Ufa!

Temos finalmente nossos ratings iniciais e já estamos prontos para atualizar os ratings em função das partidas em nossa base!

In [9]:
#EloRating2.py

df_jogosid = pd.read_csv("data/jogosid.csv")
df_teams = pd.read_csv("data/teams.csv")
df_tercas = pd.read_csv("data/tercas.csv")

datual = '2002.01.01'
df_vez = df_teams
dates = [datual]*df_vez['id'].size
df_vez = df_vez.assign(date = dates)  
df_vez = df_vez[['id','nome','pais','rate','date']]
df_todo = df_vez
df_vez.to_csv('data/rankings/rank{}.csv'.format(datual))

for date in df_tercas['2002.01.01'].values:    
    danterior = datual    
    datual = date
    df_vez = pd.read_csv('data/rankings/rank{}.csv'.format(danterior))
    entrada = list(zip(df_vez.id,df_vez.rate))
    ratings = rate_table(entrada)    
    df_jogosmes = df_jogosid[df_jogosid['date']>=danterior][df_jogosid['date']<datual]
    jogos = list(zip(df_jogosmes.id1,df_jogosmes.g1,df_jogosmes.g2,df_jogosmes.id2))
    ratings = elo_rate (ratings, jogos)        
    rates = [ratings[i][1] for i in range(len(ratings))]
    ratings=[]
    for rate in rates:
        if rate!=0:
            ratings.append(round(rate,8))
    df_vez.rate = ratings
    dates = [date]*df_vez['id'].size
    df_vez.date = dates  
    df_vez = df_vez[['id','nome','pais','rate','date']]    
    df_vez.to_csv('data/rankings/rank{}.csv'.format(datual))
    df_todo = df_todo.append(df_vez,ignore_index=True)
    print(datual)
    print(df_vez.sort_values(by=['rate'],ascending=False)[['id','nome','pais','rate']].head())
    print(df_vez.sort_values(by=['rate'],ascending=False)[['id','nome','pais','rate']].tail())

[[1, 1572.01571704], [2, 1572.01571704], [3, 1572.01571704], [4, 1572.01571704], [5, 1572.01571704], [6, 1572.01571704], [7, 1572.01571704], [8, 1572.01571704], [9, 1572.01571704], [10, 1572.01571704], [11, 1572.01571704], [12, 1572.01571704], [13, 1572.01571704], [14, 1572.01571704], [15, 1572.01571704], [16, 1572.01571704], [17, 1572.01571704], [18, 1572.01571704], [19, 1572.01571704], [20, 1572.01571704], [21, 1572.01571704], [22, 1572.01571704], [23, 1572.01571704], [24, 1572.01571704], [25, 1572.01571704], [26, 1572.01571704], [27, 1572.01571704], [28, 1572.01571704], [29, 1572.01571704], [30, 1572.01571704], [31, 1572.01571704], [32, 1572.01571704], [33, 1572.01571704], [34, 1572.01571704], [35, 1572.01571704], [36, 1572.01571704], [37, 1572.01571704], [38, 1572.01571704], [39, 1572.01571704], [40, 1572.01571704], [41, 1572.01571704], [42, 1572.01571704], [43, 0.0], [44, 0.0], [45, 0.0], [46, 0.0], [47, 0.0], [48, 0.0], [49, 0.0], [50, 0.0], [51, 0.0], [52, 0.0], [53, 0.0], [54, 

2002.02.05
      id        nome    pais         rate
73   202  America RN  Brasil  1604.499521
106  235  Santa Cruz  Brasil  1604.499521
91   220  Fluminense  Brasil  1604.499521
113  242     Vitoria  Brasil  1604.499521
112  241       Vasco  Brasil  1604.499521
      id                     nome  pais         rate
253  725              Juan Aurich  Peru  1392.867014
252  724              Jose Galvez  Peru  1392.867014
251  723                   Huaral  Peru  1392.867014
250  722                Garcilaso  Peru  1392.867014
248  720  Estudiantes de Medicina  Peru  1392.867014
[[1, 1572.01571704], [2, 1572.01571704], [3, 1572.01571704], [4, 1559.21441704], [5, 1572.01571704], [6, 1572.01571704], [7, 1572.01571704], [8, 1572.01571704], [9, 1572.01571704], [10, 1569.21441704], [11, 1572.01571704], [12, 1564.81701704], [13, 1574.81701704], [14, 1572.01571704], [15, 1572.01571704], [16, 1569.21441704], [17, 1572.01571704], [18, 1574.81701704], [19, 1572.01571704], [20, 1569.21441704], [21, 15

2002.03.05
      id         nome       pais         rate
30    31  River Plate  Argentina  1638.228347
167  421  Millonarios   Colombia  1615.721836
110  239    Sao Paulo     Brasil  1604.499521
99   228    Juventude     Brasil  1604.499521
93   222         Gama     Brasil  1604.499521
      id                nome     pais         rate
247  719      Dep. Municipal     Peru  1392.867014
246  718  Defensor La Bocana     Peru  1392.867014
268  740      Union Comercio     Peru  1392.867014
269  741               Wanka     Peru  1392.867014
53   112      Iberoamericana  Bolivia  1383.879778
[[1, 1572.01571704], [2, 1572.01571704], [3, 1572.01571704], [4, 1543.35617721], [5, 1572.01571704], [6, 1572.01571704], [7, 1572.01571704], [8, 1580.43390716], [9, 1553.48691016], [10, 1589.18455602], [11, 1572.01571704], [12, 1568.60307213], [13, 1563.32510494], [14, 1572.01571704], [15, 1572.01571704], [16, 1540.93577796], [17, 1572.01571704], [18, 1591.57532339], [19, 1572.01571704], [20, 1580.443173

      id           nome       pais         rate
156  410      Dep. Cali   Colombia  1689.421936
30    31    River Plate  Argentina  1659.260201
173  427       Santa Fe   Colombia  1654.682962
95   224         Gremio     Brasil  1648.793960
17    18  Gimnasia L.P.  Argentina  1635.295411
      id           nome       pais         rate
331  928     Portuguesa  Venezuela  1372.372681
119  306       Cobresal      Chile  1369.366514
54   113  Independiente    Bolivia  1359.955836
52   111        Guabira    Bolivia  1327.146509
57   116       Mariscal    Bolivia  1311.061840
[[1, 1572.01571704], [2, 1572.01571704], [3, 1572.01571704], [4, 1509.10436523], [5, 1572.01571704], [6, 1572.01571704], [7, 1572.01571704], [8, 1585.29390948], [9, 1508.72363031], [10, 1615.19273348], [11, 1572.01571704], [12, 1559.59109702], [13, 1573.71725705], [14, 1572.01571704], [15, 1572.01571704], [16, 1596.45660744], [17, 1572.01571704], [18, 1635.29541138], [19, 1572.01571704], [20, 1594.75571016], [21, 1572.01

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


...

2019.03.05
      id              nome       pais         rate
101  230         Palmeiras     Brasil  1908.648404
9     10      Boca Juniors  Argentina  1855.756630
217  616  Olimpia Asuncion   Paraguai  1845.621481
90   219       Flamengo RJ     Brasil  1840.912703
95   224            Gremio     Brasil  1840.488070
      id                     nome     pais         rate
251  723                   Huaral     Peru  1259.612834
244  716                  Coopsol     Peru  1255.075677
61   120              Real Mamore  Bolivia  1208.696440
248  720  Estudiantes de Medicina     Peru  1188.803190
53   112           Iberoamericana  Bolivia  1186.178290
[[1, 1566.69226995], [2, 1526.88921341], [3, 1517.61590216], [4, 1564.86101365], [5, 1513.88242929], [6, 1554.5613033], [7, 1705.2878075999997], [8, 1630.86587731], [9, 1588.73480904], [10, 1855.75663029], [11, 1549.72542698], [12, 1480.18134525], [13, 1624.24689875], [14, 1420.9276065], [15, 1777.02772935], [16, 1630.01439391], [17, 1503.927411

2019.04.02
      id          nome       pais         rate
101  230     Palmeiras     Brasil  1920.829674
9     10  Boca Juniors  Argentina  1887.353952
90   219   Flamengo RJ     Brasil  1855.536593
30    31   River Plate  Argentina  1850.989823
29    30   Racing Club  Argentina  1831.266955
      id                     nome     pais         rate
251  723                   Huaral     Peru  1259.612834
244  716                  Coopsol     Peru  1255.075677
61   120              Real Mamore  Bolivia  1208.696440
248  720  Estudiantes de Medicina     Peru  1188.803190
53   112           Iberoamericana  Bolivia  1186.178290
[[1, 1570.7500414], [2, 1535.15266716], [3, 1521.36531664], [4, 1567.2566904], [5, 1513.88242929], [6, 1550.81188882], [7, 1691.52853437], [8, 1618.50733126], [9, 1587.49457069], [10, 1887.35395154], [11, 1549.72542698], [12, 1480.18134525], [13, 1618.70034928], [14, 1420.9276065], [15, 1762.42880003], [16, 1648.64132536], [17, 1503.92741137], [18, 1578.1172589999999],

30    31       River Plate  Argentina  1846.066870
      id                     nome     pais         rate
251  723                   Huaral     Peru  1259.612834
244  716                  Coopsol     Peru  1255.075677
61   120              Real Mamore  Bolivia  1208.696440
248  720  Estudiantes de Medicina     Peru  1188.803190
53   112           Iberoamericana  Bolivia  1186.178290
[[1, 1573.38432707], [2, 1535.15266716], [3, 1521.36531664], [4, 1616.91938292], [5, 1513.88242929], [6, 1550.81188882], [7, 1652.15418904], [8, 1619.7455252999998], [9, 1598.9765906999999], [10, 1847.12115726], [11, 1556.71700035], [12, 1480.18134525], [13, 1613.88006167], [14, 1420.9276065], [15, 1729.73116119], [16, 1658.01491325], [17, 1503.92741137], [18, 1617.63711488], [19, 1680.10330255], [20, 1636.44260127], [21, 1390.47057154], [22, 1733.60630917], [23, 1476.81226382], [24, 1624.20836371], [25, 1622.9359979], [26, 1534.8739125999998], [27, 1479.6728022], [28, 1588.71564374], [29, 1479.9531811], [

[[1, 1573.38432707], [2, 1535.15266716], [3, 1521.36531664], [4, 1616.91938292], [5, 1513.88242929], [6, 1550.81188882], [7, 1652.15418904], [8, 1619.7455252999998], [9, 1598.9765906999999], [10, 1847.12115726], [11, 1556.71700035], [12, 1480.18134525], [13, 1613.88006167], [14, 1420.9276065], [15, 1729.73116119], [16, 1658.01491325], [17, 1503.92741137], [18, 1617.63711488], [19, 1680.10330255], [20, 1636.44260127], [21, 1390.47057154], [22, 1733.60630917], [23, 1476.81226382], [24, 1624.20836371], [25, 1622.9359979], [26, 1534.8739125999998], [27, 1479.6728022], [28, 1588.71564374], [29, 1479.9531811], [30, 1808.78742508], [31, 1846.06686957], [32, 1589.00864239], [33, 1662.46885065], [34, 1584.22658189], [35, 1532.95506346], [36, 1464.89767726], [37, 1663.29959153], [38, 1536.98657861], [39, 1715.5400029], [40, 1450.8120472], [41, 1688.68192218], [42, 1728.09606333], [43, 0.0], [44, 0.0], [45, 0.0], [46, 0.0], [47, 0.0], [48, 0.0], [49, 0.0], [50, 0.0], [51, 0.0], [52, 0.0], [53, 0.

2019.08.06
      id           nome       pais         rate
101  230      Palmeiras     Brasil  1929.536043
9     10   Boca Juniors  Argentina  1869.050497
30    31    River Plate  Argentina  1844.090224
95   224         Gremio     Brasil  1833.843125
97   226  Internacional     Brasil  1832.090971
      id                     nome     pais         rate
251  723                   Huaral     Peru  1266.824231
244  716                  Coopsol     Peru  1255.075677
61   120              Real Mamore  Bolivia  1208.696440
248  720  Estudiantes de Medicina     Peru  1188.803190
53   112           Iberoamericana  Bolivia  1186.178290
[[1, 1567.26633306], [2, 1535.15266716], [3, 1521.36531664], [4, 1619.4305635], [5, 1544.55433186], [6, 1550.81188882], [7, 1627.97399974], [8, 1618.8275002], [9, 1598.9765906999999], [10, 1869.05049685], [11, 1556.44113306], [12, 1480.18134525], [13, 1589.85071445], [14, 1420.9276065], [15, 1704.06750344], [16, 1654.05794971], [17, 1503.92741137], [18, 1614.3780

Pronto!
Ratings calculados!
E ainda salvamos "mensalmente" os ratings em um arquivo específico.
Vamos ver o que temos...


In [10]:
print(df_vez.sort_values(by=['rate'],ascending=False)[['id','nome','pais','rate']])

      id                     nome       pais         rate
90   219              Flamengo RJ     Brasil  1897.469094
101  230                Palmeiras     Brasil  1893.081788
9     10             Boca Juniors  Argentina  1871.235910
30    31              River Plate  Argentina  1856.919882
95   224                   Gremio     Brasil  1847.061704
217  616         Olimpia Asuncion   Paraguai  1830.055634
97   226            Internacional     Brasil  1819.198407
29    30              Racing Club  Argentina  1794.650963
108  237                   Santos     Brasil  1788.748980
85   214              Corinthians     Brasil  1780.313427
110  239                Sao Paulo     Brasil  1764.338080
286  817                 Nacional    Uruguai  1760.607022
74   203             Athletico-PR     Brasil  1758.414447
76   205              Atletico-MG     Brasil  1754.035029
205  604            Cerro Porteno   Paraguai  1753.611774
167  421              Millonarios   Colombia  1747.066581
78   207      

Por fim, vamos armazenar todos os rankings num único arquivo para facilitar a consulta pela base do nosso site.

In [11]:
df_todo = df_todo[['id','nome','pais','rate','date']]
print(df_todo)
df_todo.to_csv('data/rankings/ranktodos.csv')

        id                  nome       pais         rate        date
0        1              Aldosivi  Argentina  1572.015717  2002.01.01
1        2              All Boys  Argentina  1572.015717  2002.01.01
2        3               Almagro  Argentina  1572.015717  2002.01.01
3        4        Argentinos Jrs  Argentina  1572.015717  2002.01.01
4        5       Arsenal Sarandi  Argentina  1572.015717  2002.01.01
5        6          Atl. Rafaela  Argentina  1572.015717  2002.01.01
6        7          Atl. Tucuman  Argentina  1572.015717  2002.01.01
7        8              Banfield  Argentina  1572.015717  2002.01.01
8        9              Belgrano  Argentina  1572.015717  2002.01.01
9       10          Boca Juniors  Argentina  1572.015717  2002.01.01
10      11       Central Cordoba  Argentina  1572.015717  2002.01.01
11      12     Chacarita Juniors  Argentina  1572.015717  2002.01.01
12      13        Colon Santa FE  Argentina  1572.015717  2002.01.01
13      14     Crucero del Norte  